### Read the data from the input Excel file

In [213]:
import pandas as pd
inputFileName = "MineScheduling_smallDataSet.xlsx"
paramDF = pd.read_excel(inputFileName, "Param", skiprows=0)
jobDF = pd.read_excel(inputFileName, "Job", skiprows=0)
jobMachineDF = pd.read_excel(inputFileName, "JobMachine", skiprows=0)
switchingDF = pd.read_excel(inputFileName, "Switching", skiprows=0)

### Set the data into dict + compute maxDuration

In [214]:
horizonDuration = paramDF['Duration (day)'][0]

duration = dict()  #duration[vJob][vMachine]
maxDuration = 0.0
for iJob,vJob in enumerate(jobMachineDF['Job']):
    vMachine = jobMachineDF['Machine'][iJob]
    if not vJob in duration : duration[vJob] = dict() 
    duration[vJob][vMachine] = jobMachineDF['Duration'][iJob]
    if duration[vJob][vMachine] > maxDuration : maxDuration = duration[vJob][vMachine]


### Create your linear program

In [215]:
from pulp import *
prob = LpProblem("MineSchedulingProblem", LpMaximize)

### Add variables

In [216]:
#allocationVar[vJob][vMachine] & isFirst[vJob][vMachine]
allocationVar = dict()
isFirstVar = dict()  #Jobs that each machine can do
for iJob,vJob in enumerate(jobMachineDF['Job']):
    vMachine = jobMachineDF['Machine'][iJob]
    if not vJob in allocationVar : allocationVar[vJob] = dict() 
    if not vJob in isFirstVar : isFirstVar[vJob] = dict() 
    allocationVar[vJob][vMachine] = LpVariable("Allocation[%s][%s]"%(vJob,vMachine), cat='Binary')
    isFirstVar[vJob][vMachine] = LpVariable("isFirst[%s][%s]"%(vMachine, vJob), cat='Binary') 

#startVar[vJob] and isOver[vJob]
startVar = dict()
isOverVar = dict() # 
for vJob in jobDF['Id'] :
    startVar[vJob] = LpVariable("Start[%s]"%(vJob), lowBound=0, upBound=horizonDuration, cat='Continuous')
    isOverVar[vJob] = LpVariable("IsOver[%s]"%(vJob), lowBound=0, cat='Binary')

    
#isSuccessor[vMachine][vJob1-vJob2] = 1 if vJob2 is the direct successor of vJob1
isSuccessorVar = dict()  # successor job after the previous job is done regarding a machine 
for iMachine,vMachine in enumerate(switchingDF['Machine']):
    vJob1 = switchingDF['Job1'][iMachine]
    vJob2 = switchingDF['Job2'][iMachine]
    if not vMachine in isSuccessorVar : isSuccessorVar[vMachine] = dict()
    if not vJob1 in isSuccessorVar[vMachine] : isSuccessorVar[vMachine][vJob1] = dict() 
    if not vJob2 in isSuccessorVar[vMachine] : isSuccessorVar[vMachine][vJob2] = dict() 
    isSuccessorVar[vMachine][vJob1][vJob2] = LpVariable("IsSuccessor[%s][%s][%s]"%(vMachine, vJob1, vJob2), cat='Binary')
    isSuccessorVar[vMachine][vJob2][vJob1] = LpVariable("IsSuccessor[%s][%s][%s]"%(vMachine, vJob2, vJob1), cat='Binary')  

isFirstVar

{'Job1': {'Bull': isFirst_Bull__Job1_},
 'Job2': {'Bull': isFirst_Bull__Job2_},
 'Job3': {'Bull': isFirst_Bull__Job3_,
  'SmallDragline': isFirst_SmallDragline__Job3_},
 'Job4': {'SmallDragline': isFirst_SmallDragline__Job4_,
  'BigDragline': isFirst_BigDragline__Job4_},
 'Job5': {'SmallDragline': isFirst_SmallDragline__Job5_,
  'BigDragline': isFirst_BigDragline__Job5_},
 'Job6': {'BigDragline': isFirst_BigDragline__Job6_},
 'Job7': {'Bull': isFirst_Bull__Job7_,
  'SmallDragline': isFirst_SmallDragline__Job7_},
 'Job8': {'SmallDragline': isFirst_SmallDragline__Job8_,
  'BigDragline': isFirst_BigDragline__Job8_},
 'Job9': {'SmallDragline': isFirst_SmallDragline__Job9_,
  'BigDragline': isFirst_BigDragline__Job9_},
 'Job10': {'BigDragline': isFirst_BigDragline__Job10_},
 'Job11': {'Bull': isFirst_Bull__Job11_},
 'Job12': {'Bull': isFirst_Bull__Job12_},
 'Job13': {'Bull': isFirst_Bull__Job13_,
  'SmallDragline': isFirst_SmallDragline__Job13_},
 'Job14': {'SmallDragline': isFirst_SmallDra

### Add objective

In [217]:
Production=jobDF.set_index('Id').to_dict()['Production']  # Job & Production


obj = LpAffineExpression()
obj += lpSum([isOverVar[j] * Production[j] for j in jobDF['Id']])
prob += obj


### Add constraints

In [218]:
#Max one allocation per job
#Define a dictionary : key = Job, value = Associated machines
for vJob in jobDF['Id']:
    prob += lpSum([allocationVar[vJob][vMachine] for vMachine in allocationVar[vJob].keys()])<=1


In [219]:
#Max one "isFirst" job per
for vMachine in paramDF['Machine']:
        vJobs=jobMachineDF.loc[jobMachineDF['Machine'] == vMachine , 'Job']
        prob += lpSum([isFirstVar[vJob][vMachine] for vJob in vJobs])<=1

In [220]:
#Constraint stating the "start" variable
#Start[j’] >= Start[j] + duration[j][m] + switchingTime[m][j][j’]  - BigM * (1 - isSuccessor[m][j][j’])
#Start[j'] - Start[j] - BigM * isSuccessor[m][j][j'] >= duration[j][m] + switchingTime[m][j][j’] - BigM
BigM=100000
for iMachine,vMachine in enumerate(switchingDF['Machine']):
    Job1=str(switchingDF['Job1'][iMachine])
    Job2=str(switchingDF['Job2'][iMachine])
    duration = float(jobMachineDF.loc[(jobMachineDF['Machine']==vMachine)&(jobMachineDF['Job']==Job1),'Duration'])
    duration2 = float(jobMachineDF.loc[(jobMachineDF['Machine']==vMachine)&(jobMachineDF['Job']==Job2),'Duration'])    
    SwitchingTime = float(switchingDF['SwitchingTime (day)'][iMachine])
    #print(Job1)
    #print(Job2)
    #print(duration)
    #print(SwitchingTime)
    prob+=startVar[Job2]>=startVar[Job1] + duration + SwitchingTime-BigM*(1-isSuccessorVar[vMachine][Job1][Job2])
    prob+=startVar[Job1]>=startVar[Job2] + duration2 + SwitchingTime-BigM*(1-isSuccessorVar[vMachine][Job2][Job1])

In [221]:
#Constraint stating the "isOver" variable
#start[j] + Sum[m in isPossible[j] ] duration[j][m] * allocation[j][m] <= durationHorizon + BigM * (1 - isOver[j])
#start[j] + Sum[m in isPossible[j] ] duration[j][m] * allocation[j][m] + BigM * isOver[j] <= durationHorizon + BigM

for vJob in jobDF['Id']:
    duration = jobMachineDF[jobMachineDF['Job']==vJob]
    durationdict=dict(zip(duration.Machine, duration.Duration))
    prob+=startVar[vJob]+lpSum([durationdict[vMachine]*allocationVar[vJob][vMachine] for vMachine in durationdict.keys()])<=horizonDuration+BigM*(1-isOverVar[vJob])

In [222]:
#Define a dictionary : key = Job, value = Associated machines
MachineJob_dict = dict(jobMachineDF.groupby('Job')['Machine'].apply(list))

In [223]:
#Constraint stating that if isOver = 1, then at least one allocation = 1
for vJob in jobDF['Id']:
    prob+=  isOverVar[vJob] <= lpSum(allocationVar[vJob][vMachine] for vMachine in MachineJob_dict[vJob] )

In [224]:
#Max one successor per job
for iJob,vJob in enumerate(jobMachineDF['Job']):
      prob += lpSum ( isSuccessorVar[vMachine][vJob][oJob] for vMachine in MachineJob_dict[vJob] for oJob in isSuccessorVar[vMachine][vJob] )<= 1

In [225]:
#other constraint

In [226]:
machine = list(paramDF['Machine'])
machine = paramDF[['Machine']]
machine_names = machine.Machine.unique()
Jobmachine_dict=dict(jobMachineDF.groupby('Machine')['Job'].apply(list))   

In [227]:
#Constraint stating that IsSuccessor[m][j1][j2] = 1 --> allocation[j1][m] = 1
for i in range(len(switchingDF['Machine'])):
    vMachine=switchingDF['Machine'][i]
    Job1=switchingDF['Job1'][i]
    Job2=switchingDF['Job2'][i]
    prob += isSuccessorVar[vMachine][Job1][Job2] <= allocationVar[Job1][vMachine]


In [228]:
#Constraint stating that if start > 0, then then at least one allocation = 1
for vJob in jobDF['Id']:
    prob += startVar[vJob] <= BigM * lpSum(allocationVar[vJob][vMachine] for vMachine in MachineJob_dict[vJob] )

In [229]:
#If Sum allocation = 1, then start >= 1.0
for vJob in jobDF['Id']:
    prob +=  lpSum(allocationVar[vJob][vMachine] for vMachine in MachineJob_dict[vJob]) <= startVar[vJob] 

In [230]:
## Constraint imposing predecessors : isOver[j] = 1  isOver[vPred] = 1  
jPred_dict=dict(zip(jobDF['Id'], jobDF['PredecessorId'])) 
for j in jobDF['Id']:
    if jPred_dict[j] in jobDF['Id']:
        prob +=  isOverVar[j] <= isOverVar[jPred_dict[j]]
    continue

In [231]:
 # Constraint imposing predecessors :   j shall start only once vPred has finished
for j in jobDF['Id']:
    jPred=jPred_dict[j]
    if jPred not in jobDF['Id']:
        continue
    prob += StartVar[jPred] + lpSum(duration[jPred][m] * allocationVar[jPred][m] for m in MachineJ_dict[jPred])<= StartVar[j]+BigM*(1-lpSum([allocationVar[j][m] for m in MachineJ_dict[jPred]]))

### Solve and display results

In [232]:
prob.writeLP("MineSchedulingProblem", writeSOS=1, mip=1)
prob.solve()
print("Status:", LpStatus[prob.status])
print ("Objective = ", value(prob.objective))
varsDict = {}
for v in prob.variables():
    varsDict[v.name] = v.varValue
    if "IsOver" in v.name or "Start" in v.name or "Allocation" in v.name or "IsSuccessor" in v.name:
        if v.varValue != 0.0 : print(v.name, "=", v.varValue)

Status: Optimal
Objective =  882.0
Allocation_Job10__BigDragline_ = 1.0
Allocation_Job11__Bull_ = 1.0
Allocation_Job12__Bull_ = 1.0
Allocation_Job13__SmallDragline_ = 1.0
Allocation_Job14__SmallDragline_ = 1.0
Allocation_Job15__BigDragline_ = 1.0
Allocation_Job16__BigDragline_ = 1.0
Allocation_Job17__SmallDragline_ = 1.0
Allocation_Job18__BigDragline_ = 1.0
Allocation_Job19__BigDragline_ = 1.0
Allocation_Job1__Bull_ = 1.0
Allocation_Job2__Bull_ = 1.0
Allocation_Job3__SmallDragline_ = 1.0
Allocation_Job4__SmallDragline_ = 1.0
Allocation_Job5__BigDragline_ = 1.0
Allocation_Job6__BigDragline_ = 1.0
Allocation_Job7__SmallDragline_ = 1.0
Allocation_Job8__SmallDragline_ = 1.0
Allocation_Job9__SmallDragline_ = 1.0
IsOver_Job10_ = 1.0
IsOver_Job11_ = 1.0
IsOver_Job12_ = 1.0
IsOver_Job13_ = 1.0
IsOver_Job14_ = 1.0
IsOver_Job15_ = 1.0
IsOver_Job16_ = 1.0
IsOver_Job17_ = 1.0
IsOver_Job18_ = 1.0
IsOver_Job19_ = 1.0
IsOver_Job1_ = 1.0
IsOver_Job2_ = 1.0
IsOver_Job3_ = 1.0
IsOver_Job4_ = 1.0
IsOver_